In [46]:
!pip install keras-tuner

     |████████████████████████████████| 135 kB 25.5 MB/s eta 0:00:01


In [38]:
pwd

'/home/wsuser/work'

In [39]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-0a54def8-ba7a-4324-86c3-91a7d4e0f076',
    'IBM_API_KEY_ID': 'QCDLqdfd2xk4nas6XFp-lGuleeLyqfOx-uIfCNWkM9Wg',
    'ENDPOINT': 'https://s3.private.eu.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'forestfiredetection-donotdelete-pr-xhu8yulw9tk7x9',
    'FILE': '1900.zip'
}


In [40]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
                      ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                      ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                      ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                      config=Config(signature_version='oauth'),
                      endpoint_url=credentials_1['ENDPOINT'])

In [41]:
cos.download_file(Bucket=credentials_1['BUCKET'],Key='ffd.zip',Filename='ffd.zip')

In [70]:
from zipfile import ZipFile

print('Extract all files in zip to curent directory')
with ZipFile('ffd.zip','r') as zipObj:
    zipObj.extractall()

Extract all files in zip to curent directory


In [94]:
import os
os.listdir()

['ffd.zip', '.virtual_documents', 'content']

In [93]:
import shutil
shutil.rmtree('my_project')

In [48]:
#To define linear intialisation import Sequential 
from keras.models import Sequential
#To add Layers import Dense
from keras.layers import Dense
#To create Convolution kernel import convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPooling2D
#import Flatten Layer
from keras.layers import Flatten
from keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

In [87]:
import keras  
from keras.preprocessing.image import ImageDataGenerator
def getdata():
    train_datagen=ImageDataGenerator (rescale=1./255,
                                        shear_range=0.2,
                                        rotation_range=180,
                                        zoom_range=0.2,
                                        horizontal_flip=True)

    test_datagen=ImageDataGenerator (rescale=1./255)

    test_path = "content/drive/MyDrive/Forest Fire Dataset/Testing"

    train_path = "content/drive/MyDrive/Forest Fire Dataset/Training"

    x_train = train_datagen.flow_from_directory(train_path, 
                                                target_size = (128,128), 
                                                batch_size = 32, 
                                                class_mode= 'categorical')

    x_test = test_datagen.flow_from_directory(test_path, 
                                                target_size = (128,128), 
                                                batch_size = 32, 
                                                class_mode= 'categorical')
    return x_train, x_test

In [88]:
x_train, x_test = getdata()

Found 1530 images belonging to 2 classes.
Found 380 images belonging to 2 classes.


In [90]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=2),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5,7]),
        #activation function
        activation='relu',
        input_shape=(128,128,3)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=2),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5,7]),
        #activation function
        activation='relu'
    ),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=64, step=2),
        activation='relu'
    ),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_2_units', min_value=64, max_value=128, step=2),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(2, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.adam_v2.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [91]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True)

In [95]:
from keras_tuner import BayesianOptimization
#creating Bayesiansearch object
tuner = BayesianOptimization(build_model,
                    objective='val_accuracy',
                    max_trials = 30,
                    executions_per_trial=1,
                    overwrite=True,
                    directory='my_project')
# search best parameter
tuner.search(x_train,epochs=3,validation_data=(x_test)) 

Trial 3 Complete [00h 17m 52s]
val_accuracy: 0.9394736886024475

Best val_accuracy So Far: 0.9394736886024475
Total elapsed time: 00h 35m 37s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
128               |32                |conv_1_filter
3                 |3                 |conv_1_kernel
128               |128               |conv_2_filter
7                 |7                 |conv_2_kernel
32                |64                |dense_1_units
64                |64                |dense_2_units
0.001             |0.001             |learning_rate

Epoch 1/3
48/48 [==============================] - 1235s 26s/step - loss: 1.4159 - accuracy: 0.8327 - val_loss: 0.2099 - val_accuracy: 0.9342
Epoch 2/3
48/48 [==============================] - 1196s 25s/step - loss: 0.1939 - accuracy: 0.9294 - val_loss: 0.1887 - val_accuracy: 0.9421
Epoch 3/3
 4/48 [=>............................] - ETA: 17:05 - loss: 0.2552 - accuracy: 0.9219

KeyboardInterrupt: 

In [96]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(128,128,3))
best_model.summary()

2022-09-30 15:12:57.758373: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_11 in the registry.
Traceback (most recent call last):

  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 263, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_11 in the registry.


2022-09-30 15:12:57.758489: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_11 in the registry.
Traceback (most recent call last):

  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 263, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_11 in t

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 120, 128)     200832    
                                                                 
 flatten (Flatten)           (None, 1843200)           0         
                                                                 
 dense (Dense)               (None, 64)                117964864 
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 118,170,882
Trainable params: 118,170,882


In [ ]:
from keras import callbacks
#Training the model 
history = best_model.fit_generator(x_train, steps_per_epoch=len(x_train), 
                    epochs=30, validation_data=x_test,
                    callbacks = [es],
                    validation_steps=len(x_test))

#save the model
best_model.save("forest1.h5")

Epoch 1/30
48/48 [==============================] - 361s 7s/step - loss: 0.2067 - accuracy: 0.9255 - val_loss: 0.2085 - val_accuracy: 0.9342
Epoch 2/30
48/48 [==============================] - 361s 8s/step - loss: 0.1941 - accuracy: 0.9399 - val_loss: 0.1711 - val_accuracy: 0.9526
Epoch 3/30
 1/48 [..............................] - ETA: 5:52 - loss: 0.0526 - accuracy: 1.0000

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
img = image.load_img('content/drive/MyDrive/Forest Fire Dataset/Testing.jpg',target_size=(128,128))
img

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x

In [ ]:
best_model.predict(x)

In [ ]:
xtrain.class_indices

In [ ]:
op = ['','']
pred = np.argmax(model.predict(x))
op[pred]

In [ ]:

print("Accuracy of our model on test data : " , best_model.evaluate(x_test)[1]*100 , "%")

epochs = [i for i in range(50)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']


ax[0].plot(epochs , train_loss , label = 'Training Loss')
ax[0].plot(epochs , test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()